In [1]:
from pydantic import BaseModel, Field
from typing import List
import instructor
from openai import OpenAI
from langchain_core.messages import AIMessage, ToolMessage

import json

from typing import List, Dict, Any, Annotated, Optional
from operator import add
from jinja2 import Template

#### Coordinator Agent

In [2]:
class Delegation(BaseModel):
    agent: str
    task: str = Field(default="")


class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: list[Delegation]
    final_answer: bool = Field(default=False)
    answer: str


class MCPToolCall(BaseModel):
    name: str
    arguments: dict
    server: str

class ToolCall(BaseModel):
    name: str
    arguments: dict


class RAGUsedContext(BaseModel):
    id: str
    description: str


class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    answer: str = ""
    product_qa_iteration: int = Field(default=0)
    shopping_cart_iteration: int = Field(default=0)
    coordinator_iteration: int = Field(default=0)
    product_qa_final_answer: bool = Field(default=False)
    shopping_cart_final_answer: bool = Field(default=False)
    coordinator_final_answer: bool = Field(default=False)
    product_qa_available_tools: List[Dict[str, Any]] = []
    shopping_cart_available_tools: List[Dict[str, Any]] = []
    mcp_tool_calls: Optional[List[MCPToolCall]] = Field(default_factory=list)
    tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    retrieved_context_ids: List[RAGUsedContext] = []
    trace_id: str = ""
    user_id: str = ""
    cart_id: str = ""
    plan: list[Delegation] = Field(default_factory=list)
    next_agent: str = ""

In [3]:
def lc_messages_to_regular_messages(msg):

    if isinstance(msg, dict):
        
        if msg.get("role") == "user":
            return {"role": "user", "content": msg["content"]}
        elif msg.get("role") == "assistant":
            return {"role": "assistant", "content": msg["content"]}
        elif msg.get("role") == "tool":
            return {
                "role": "tool", 
                "content": msg["content"], 
                "tool_call_id": msg.get("tool_call_id")
            }
        
    elif isinstance(msg, AIMessage):

        result = {
            "role": "assistant",
            "content": msg.content
        }
        
        if hasattr(msg, 'tool_calls') and msg.tool_calls and len(msg.tool_calls) > 0 and not msg.tool_calls[0].get("name").startswith("functions."):
            result["tool_calls"] = [
                {
                    "id": tc["id"],
                    "type": "function",
                    "function": {
                        "name": tc["name"].replace("functions.", ""),
                        "arguments": json.dumps(tc["args"])
                    }
                }
                for tc in msg.tool_calls
            ]
            
        return result
    
    elif isinstance(msg, ToolMessage):

        return {"role": "tool", "content": msg.content, "tool_call_id": msg.tool_call_id}
    
    else:

        return {"role": "user", "content": str(msg)}

In [4]:
def coordinator_agent_node(state) -> dict:

    prompt_template = """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- product_qa_agent: The user is asking a question about a product. This can be a question about available products, their specifications, user reviews etc.
- shopping_cart_agent: The user is asking to add or remove items from the shopping cart or questions about the current shopping cart.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true

Additional instructions:

- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

    prompt = Template(prompt_template).render()

    messages = state.messages

    conversation = []

    for msg in messages:
        conversation.append(lc_messages_to_regular_messages(msg))

    client = instructor.from_openai(OpenAI())

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1",
        response_model=CoordinatorAgentResponse,
        messages=[{"role": "system", "content": prompt}, *conversation],
        temperature=0,
    )

    if response.final_answer:
        ai_message = [AIMessage(
            content=response.answer,
        )]
    else:
        ai_message = []

    return {
        "messages": ai_message,
        "answer": response.answer,
        "next_agent": response.next_agent,
        "plan": response.plan,
        "coordinator_final_answer": response.final_answer,
        "coordinator_iteration": state.coordinator_iteration + 1,
        "trace_id": ""
    }

In [5]:
initial_state = State(messages=[{"role": "user", "content": "What is the weather today?"}])

In [6]:
answer = coordinator_agent_node(initial_state)

In [7]:
answer

{'messages': [AIMessage(content="I'm here to assist you with shopping-related queries, such as finding products, answering questions about items, or helping with your shopping cart. I don't have access to weather information. If you need help with shopping, please let me know how I can assist you!", additional_kwargs={}, response_metadata={})],
 'answer': "I'm here to assist you with shopping-related queries, such as finding products, answering questions about items, or helping with your shopping cart. I don't have access to weather information. If you need help with shopping, please let me know how I can assist you!",
 'next_agent': '',
 'plan': [],
 'coordinator_final_answer': True,
 'coordinator_iteration': 1,
 'trace_id': ''}

### Coordinator Evaluation Dataset

In [8]:
coordinator_eval_dataset = [
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Whats is the weather today?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can I get some earphones?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add an item with ID B09NLTDHQ6 to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add those earphones to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add the best items to my cart? I am looking for laptop bags."}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you find some good reviews for items in my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you put the items with the most positive user reviews to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What kind of stuff do you sell?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you help me with my order?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add two, ideally red tablets to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    }
]

#### Upload the dataset to LangSmith

In [9]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

dataset_name = "coordinator-evaluation-dataset-01"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating routing of the coordinator agent"
)

In [10]:
for item in coordinator_eval_dataset:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"messages": item["inputs"]["messages"]},
        outputs={
            "next_agent": item["outputs"]["next_agent"],
            "coordinator_final_answer": item["outputs"]["coordinator_final_answer"]
        }
    )

### Run Evaluation

In [11]:
from langsmith import Client
import os

ls_client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

In [12]:
def next_agent_evaluator(run, example):

    next_agent_match = run.outputs["next_agent"] == example.outputs["next_agent"]
    final_answer_match = run.outputs["coordinator_final_answer"] == example.outputs["coordinator_final_answer"]

    return next_agent_match and final_answer_match

In [13]:
results = ls_client.evaluate(
    lambda x: coordinator_agent_node(State(messages=x["messages"])),
    data="coordinator-evaluation-dataset-01",
    evaluators=[
        next_agent_evaluator
    ],
    experiment_prefix="coordinator-evaluation-dataset-01"
)

/Users/aurimasgriciunas/AI-Engineering-Bootcamp/sprint-06-ai-engineering-bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'coordinator-evaluation-dataset-01-a39d27af' at:
https://smith.langchain.com/o/50934075-c83e-54d3-b52b-cfc91086865f/datasets/b8ad511b-ea12-458f-b4cb-7666d0090c9f/compare?selectedSessions=fbad218f-7636-4cfc-9248-40e8e0ef4a59




10it [00:17,  1.73s/it]
